In [1]:
import azureml.core
print(f'Azure ML Version {azureml.core.VERSION}')

Azure ML Version 1.16.0


In [3]:
from azureml.core.authentication import InteractiveLoginAuthentication
credentials = InteractiveLoginAuthentication()
from azureml.core import Workspace
ws = Workspace.from_config(auth=credentials)

In [4]:
from azureml.core import Experiment
exp=Experiment(workspace=ws, name='Titanic-automl')

In [7]:
import pandas
train_df = pandas.read_csv('train.csv')

In [8]:
azureml.train.automl.utilities.get_primary_metrics('classification')

['AUC_weighted',
 'norm_macro_recall',
 'accuracy',
 'average_precision_score_weighted',
 'precision_score_weighted']

In [13]:
from azureml.train.automl import AutoMLConfig

config = AutoMLConfig(
    task='classification',
    primary_metric='accuracy',
    iterations=10,
    training_data=train_df,
    label_column_name='Survived',
    n_cross_validations=5,
    featurization="auto",
    model_explainability=True
)

In [14]:
run = exp.submit(config, show_output=True)

Running on local machine
Parent Run ID: AutoML_5de99f57-2f39-4574-ae21-a7be823d11a4

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

In [15]:
best_run, fitted_model = run.get_output()

In [16]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Titanic-automl,AutoML_5de99f57-2f39-4574-ae21-a7be823d11a4_8,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    max_leaf_nodes=None,
                                                                                                    max_samples=None,
                                                                                                    min_impurity_decrease=0.0,
                          

In [20]:
test = pandas.read_csv('test.csv')
output = pandas.DataFrame({
    'PassengerId': test['PassengerId'], 
    'Survived': fitted_model.predict(test)
})


In [22]:
output.to_csv("submission.csv", index=False)

In [26]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0]+ ' - ')
        elif hasattr(step[1], '_base_learners') and hasattr(step[1], '_meta_learner'):
            print("\nMeta Learner")
            pprint(step[1]._meta_learner)
            print()
            for estimator in step[1]._base_learners:
                print_model(estimator[1], estimator[0]+ ' - ')
        else:
            pprint(step[1].get_params())
            print()
            
print_model(fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['1', '0', '4', '7', '5', '2'],
 'weights': [0.1, 0.3, 0.1, 0.3, 0.1, 0.1]}

1 - maxabsscaler
{'copy': True}

1 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'tree_method': 'auto',
 'verbose': -10,
 'verbo